In [2]:
%pip install torch

ERROR: Could not find a version that satisfies the requirement torch (from versions: none)
ERROR: No matching distribution found for torch
Note: you may need to restart the kernel to use updated packages.


In [2]:
# The models the authors used:
try:
    from transformers import BertForMaskedLM, BertTokenizer, AdamW, get_linear_schedule_with_warmup
    from transformers import AlbertForMaskedLM, AlbertTokenizer
except ModuleNotFoundError:
    %pip install transformers
    from transformers import BertForMaskedLM, BertTokenizer, AdamW, get_linear_schedule_with_warmup
    from transformers import AlbertForMaskedLM, AlbertTokenizer


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 8.3 MB/s eta 0:00:00
  Using cached filelock-3.13.1-py3-none-any.whl.metadata (2.8 kB)
  Using cached numpy-1.26.2-cp312-cp312-macosx_11_0_arm64.whl.metadata (61 kB)
  Using cached PyYAML-6.0.1-cp312-cp312-macosx_11_0_arm64.whl.metadata (2.1 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 3.7 MB/s eta 0:00:00
  Using cached requests-2.31.0-py3-none-any.whl.metadata (4.6 kB)
  Using cached tokenizers-0.15.0-cp312-cp312-macosx_11_0_arm64.whl.metadata (6.7 kB)
  Using cached tqdm-4.66.1-py3-none-any.whl.metadata (57 kB)
  Using cached charset_normalizer-3.3.2-cp312-cp312-macosx_11_0_arm64.whl.metadata (33 kB)
  Using cached urllib3-2.1.0-py3-none-any.whl.metadata (6.4 kB)
  Using cached certifi-2023.11.17-py3-none-any.whl.metadata (2.2 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 12.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.1/330.1 kB 15.6 MB/s eta 0:00:00
Using cach

/Users/nazanin/.local/share/virtualenvs/BLANC_repo-_IGNgo0f/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [1]:
def BLANC_help(summary, text, model, M=6, L_min=4, sep=''):
    """
    Calculate BLANC similarity between summary and text using a specified model.

    Parameters:
    - summary (str): The summary text.
    - text (List[List[str]]): List of sentences represented as a list of words.
    - model: BERT model type
    - M (int): Parameter M for the algorithm (default is 6).
    - L_min (int): Minimum length requirement for masked words (default is 4).
    - sep (str): Separator between the inference help (filler/summary) and a sentence from the text (default is '').

    Returns:
    - float: BLANC similarity score.
    """

    filler = '.' * len(summary)
    S = [[0, 0], [0, 0]]

    for sentence in text:
        for i in range(1, M + 1):
            masked_sentence = ['<MASK>' if (j - i) % M == 0 and len(sentence[j]) >= L_min else sentence[j] 
                        for j in range(len(sentence))]
            input_base = filler + sep + masked_sentence
            input_help = summary + sep + masked_sentence
            out_base = model(input_base)
            out_help = model(input_help)
            masked_tokens = [idx for idx, word in enumerate(masked_sentence) if word == '<MASK>']

            for j in masked_tokens:
                k = int(out_base[j] == sentence[j])
                m = int(out_help[j] == sentence[j])
                S[k][m] += 1

    return (S[0][1] - S[1][0]) / (S[0][0] + S[1][1] + S[0][1] + S[1][0])



Ideas for improvement:
1. add error handling to ensure that the input to the function (summary, text, model) are of the expected types or shapes